In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
# Set OMP_NUM_THREADS to avoid memory leak warning
os.environ['OMP_NUM_THREADS'] = '1'

# Load prediction datasets from FF_Predictions folder
predictions_path = 'FF_Predictions'
prediction_files = {
    'K': os.path.join(predictions_path, 'K_predictions.csv'),
    'QB': os.path.join(predictions_path, 'QB_predictions.csv'),
    'RB': os.path.join(predictions_path, 'RB_predictions.csv'),
    'TE': os.path.join(predictions_path, 'TE_predictions.csv'),
    'WR': os.path.join(predictions_path, 'WR_predictions.csv')
}

# Load and combine prediction data
dfs = []
for position, file_path in prediction_files.items():
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df['Position'] = position
        dfs.append(df)

combined_predictions = pd.concat(dfs, ignore_index=True)

# Select relevant features for clustering
features = [
    'Games_Played',
    'sample_weight',
    'predicted_points'
]

# Additional position-specific features
position_features = {
    'QB': ['Yds_avg', 'TD_avg', 'Int_avg', 'Rate_avg', 'FL_avg'],
    'RB': ['Yds_avg', 'TD_avg', 'Rec_avg', 'FL_avg'],
    'WR': ['Rec_avg', 'Yds_avg', 'TD_avg', 'FL_avg'],
    'TE': ['Rec_avg', 'Yds_avg', 'TD_avg', 'FL_avg'],
    'K': ['FG%_avg', 'XP%_avg', 'Pts_avg']
}

# Create separate clustering analyses for each position
for position in prediction_files.keys():
    print(f"\nProcessing {position}...")
    
    position_df = combined_predictions[combined_predictions['Position'] == position].copy()
    position_df = position_df.reset_index(drop=True)
    
    # Get relevant features for this position
    current_features = features + position_features.get(position, [])
    available_features = [f for f in current_features if f in position_df.columns]
    
    if len(position_df) < 2:
        print(f"Insufficient data for position {position}")
        continue
        
    # Scale the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(position_df[available_features])
    
    # Apply KMeans clustering
    n_clusters = min(len(position_df), 3)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    position_df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Print cluster analysis
    print(f"\nCluster Analysis for {position}:")
    for cluster in sorted(position_df['cluster'].unique()):
        cluster_data = position_df[position_df['cluster'] == cluster]
        print(f"\nCluster {cluster}:")
        print(f"Average predicted points: {cluster_data['predicted_points'].mean():.2f}")
        print(f"Range: {cluster_data['predicted_points'].min():.2f} - {cluster_data['predicted_points'].max():.2f}")
        print(f"Number of players: {len(cluster_data)}")
        print("Players:", ', '.join(sorted(cluster_data['Player'].tolist())))
        
        # Print average stats for the cluster
        print("\nCluster Averages:")
        for feature in available_features:
            print(f"{feature}: {cluster_data[feature].mean():.2f}")

print("\nClustering analysis complete.")


Processing K...

Cluster Analysis for K:

Cluster 0:
Average predicted points: 22.73
Range: 19.58 - 25.32
Number of players: 8
Players: Brandon McManus, Cairo Santos, Chase McLaughlin, Eddy Piñeiro, Graham Gano, Jake Elliott, Jason Myers, Younghoe Koo

Cluster Averages:
Games_Played: 125.75
sample_weight: 1.00
predicted_points: 22.73
FG%_avg: 72.43
XP%_avg: 82.68
Pts_avg: 6.72

Cluster 1:
Average predicted points: 5.75
Range: 1.10 - 9.67
Number of players: 4
Players: Austin Seibert, Chad Ryland, John Parker Romo, Joshua Karty

Cluster Averages:
Games_Played: 19.25
sample_weight: 0.45
predicted_points: 5.75
FG%_avg: 48.72
XP%_avg: 52.59
Pts_avg: 4.83

Cluster 2:
Average predicted points: 8.75
Range: 3.06 - 11.92
Number of players: 4
Players: Blake Grupe, Brandon Aubrey, Jake Bates, Jake Moody

Cluster Averages:
Games_Played: 22.75
sample_weight: 0.57
predicted_points: 8.75
FG%_avg: 77.28
XP%_avg: 88.75
Pts_avg: 7.88

Processing QB...

Cluster Analysis for QB:

Cluster 0:
Average predic

C:\Users\donoh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\donoh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\donoh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



Cluster Analysis for RB:

Cluster 0:
Average predicted points: 1.92
Range: 0.43 - 5.46
Number of players: 18
Players: Avery Williams, Cam Akers, Craig Reynolds, DeeJay Dallas, Emari Demercado, Eric Gray, Hunter Luepke, Jeremy McNichols, Jordan Mason, Kenneth Gainwell, Raheem Blackshear, Rico Dowdle, Ronnie Rivers, Roschon Johnson, Travis Homer, Ty Chandler, Tyler Allgeier, Zach Charbonnet

Cluster Averages:
Games_Played: 41.06
sample_weight: 0.82
predicted_points: 1.92
Yds_avg: 13.31
TD_avg: 0.09
Rec_avg: 0.75
FL_avg: 0.02

Cluster 1:
Average predicted points: 0.22
Range: 0.00 - 0.64
Number of players: 15
Players: Blake Corum, Chris Brooks, Chris Rodriguez, Cody Schrader, Deuce Vaughn, Emanuel Wilson, Isaac Guerendo, Jase McClellan, Jonathon Brooks, Jordan Mims, Kenny McIntosh, Sean Tucker, Sione Vaki, Trey Benson, Will Shipley

Cluster Averages:
Games_Played: 11.20
sample_weight: 0.27
predicted_points: 0.22
Yds_avg: 6.96
TD_avg: 0.02
Rec_avg: 0.19
FL_avg: 0.01

Cluster 2:
Average pre

C:\Users\donoh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\donoh\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
